# Real-World Use Case: Mobile Price Segmentation

## 1. The Problem
A phone manufacturer wants to create a new phone. They need to know what "Price Tier" (Wait, 0=Low, 1=Medium, 2=High, 3=Flagship) their proposed specs would land in compared to competitors.

## 2. Why KNN?
*   **Similarity Principle**: Phones in the same price range often share similar specs (similar RAM, similar battery, similar pixel density).
*   **Non-Linear**: The boundary between "High" and "Flagship" might be complex and not easily separable by a straight line.

## 3. Feature Scaling (Crucial for KNN)
KNN calculates distance. `RAM` (e.g., 8000 MB) is huge compared to `Battery` (4000 mAh) or `Cores` (8). If we don't scale, RAM will dominate the distance calculation.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, confusion_matrix

# 1. Generate Data (Proxy for GSMArena dataset)
np.random.seed(42)
n = 1000
ram = np.random.randint(512, 12000, n)
battery = np.random.randint(2000, 6000, n)
px_height = np.random.randint(500, 2000, n)

# Logic for Price Range (0, 1, 2, 3)
score = (ram * 0.5) + (battery * 0.3) + (px_height * 0.2)
price_range = pd.qcut(score, q=4, labels=[0, 1, 2, 3])

df = pd.DataFrame({'RAM': ram, 'Battery': battery, 'PxHeight': px_height, 'PriceRange': price_range})

# 2. Preprocessing
X = df.drop('PriceRange', axis=1)
y = df['PriceRange']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Scale! MinMax is good for Image/Pixel data or bounded specs
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 3. Train KNN
knn = KNeighborsClassifier(n_neighbors=9)
knn.fit(X_train_scaled, y_train)

# 4. Evaluate
y_pred = knn.predict(X_test_scaled)
print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

# 5. Prediction
# New Phone: 8GB RAM (8192), 5000mAh, 1080p
new_phone = [[8192, 5000, 1080]]
new_phone_scaled = scaler.transform(new_phone)
print(f"\nPredicted Price Tier: {knn.predict(new_phone_scaled)[0]}")